In [ ]:
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

from snowflake.snowpark.session import Session
from snowflake.snowpark.context import get_active_session

# Misc
from datetime import timedelta
import json
import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)


session = get_active_session()

In [ ]:
raw_user_sends = session.table("JING_TEST_DB.RAW.USER_SENDS")
raw_user_impressions = session.table("JING_TEST_DB.RAW.USER_IMPRESSIONS")
raw_user_clicks = session.table("JING_TEST_DB.RAW.USER_CLICKS")
raw_user_purchases = session.table("JING_TEST_DB.RAW.USER_PURCHASES")

In [ ]:
agg_user_sends = (raw_user_sends
                 .group_by("USER_ID").agg(F.count("*").alias("Number_sends")))
agg_user_impressions = (raw_user_impressions
                 .group_by("USER_ID").agg(F.count("*").alias("Number_impressions")))
agg_user_clicks = (raw_user_clicks
                 .group_by("USER_ID").agg(F.count("*").alias("Number_clicks")))
agg_user_purchases = (raw_user_purchases
                 .group_by("USER_ID").agg(F.count("*").alias("Number_purchases")))


agg_user_sends.show(1)
agg_user_impressions.show(1)
agg_user_clicks.show(1)
agg_user_purchases.show(1)

In [ ]:
agg_user_events = (agg_user_sends
                  .join(agg_user_impressions, "USER_ID", "outer")
                  .join(agg_user_clicks, "USER_ID", "outer")
                  .join(agg_user_purchases, "USER_ID", "outer"))

agg_user_events.show(100)

In [ ]:
agg_user_events.write.mode("overwrite").save_as_table("JING_TEST_DB.TRANSFORM.AGGREGATED_USER_EVENTS")

In [ ]:
load_agg_user_events = session.table("JING_TEST_DB.TRANSFORM.AGGREGATED_USER_EVENTS")

agg_user_events.count(), load_agg_user_events.count()